In [1]:
%load_ext autoreload
%autoreload 2
%autoawait trio
%matplotlib inline

%load_ext tensorboard

import trio
import pyfuse3
import src
import logging
import matplotlib
import matplotlib.pyplot as plt
import IPython

from src.runner import Runner
from src.mock_runner import MockRunner
from src.env import Env
from src.stats.post import *
from src.predicting import make_predictor
import src.methods
import pandas as pd
from pandas import read_csv
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_colwidth', 500)

from sys import argv
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

import numpy as np
import json
from collections import defaultdict
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, Concatenate, Embedding, Dropout
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import model_from_json
from tensorflow.keras.utils import to_categorical
import random
import pickle
from pandas import read_csv, DataFrame
from sklearn.model_selection import train_test_split
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.debugging.set_log_device_placement(False)

GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
df = read_csv('traces/simple2')
df[0:10]

,Unnamed: 0,kind,filename,timestamp,offset,length,real_length,hit,pid,source,fingerprint
0,0,Miss,./sources/simple2/file_GkkrnO25,0.000000,0,16384,15360,False,452414,read,NaN
1,1,Miss,./sources/simple2/file_6hZMFmIB,0.001107,0,16384,15360,False,452414,read,NaN
2,2,Miss,./sources/simple2/file_0b3IZ4FK,0.001779,0,16384,15360,False,452414,read,NaN
3,3,Miss,./sources/simple2/file_pcjXb3fg,0.002563,0,16384,15360,False,452414,read,NaN
4,4,Miss,./sources/simple2/file_ZevludPw,0.003774,0,16384,15360,False,452414,read,NaN
5,5,Miss,./sources/simple2/file_19PpmxuL,0.005093,0,16384,15360,False,452414,read,NaN
6,6,Miss,./sources/simple2/file_5cgVW8xW,0.006049,0,16384,15360,False,452414,read,NaN
7,7,Miss,./sources/simple2/file_HzgKMSH9,0.006638,0,16384,15360,False,452414,read,NaN
8,8,Miss,./sources/simple2/file_DqDN7NDE,0.007442,0,16384,15360,False,452414,read,NaN
9,9,Miss,./sources/simple2/file_9ZFr6ZIC,0.009055,0,16384,15360,False,452414,read,NaN


In [5]:
from src.methods.lstm import Lstm1 as NN
nn = NN()
df = df[df.kind != 'Prefetch']
(X, Y) = nn.transform_data(df)
nn.run(X, Y, X, Y, params={ "epochs": 60, 'hidden_neurons': 400 }, verbose=0)

In [15]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer(char_level=True)
tk.fit_on_texts(['./sources/simple1/file', './sources/simple2/xdwqdqw'])
tk.texts_to_sequences(['./sources/simple1/file', './sources/simple2/xdwqdqwxxxx'])

[[6, 1, 2, 7, 8, 9, 10, 3, 2, 1, 2, 4, 11, 12, 5, 3, 16, 1, 17, 4, 5, 3],
 [6,
  1,
  2,
  7,
  8,
  9,
  10,
  3,
  2,
  1,
  2,
  4,
  11,
  12,
  5,
  3,
  18,
  1,
  19,
  13,
  14,
  15,
  13,
  15,
  14,
  19,
  19,
  19,
  19]]

In [18]:
from src.methods.utils import bag_inputs
xd = bag_inputs(['./sources/simple1/file', './sources/simple2/xdwqdqw'])

SyntaxError: unexpected EOF while parsing (<ipython-input-18-311345e1b2da>, line 2)